<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP_11_BERTopic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial** - Topic Modeling with BERTopic
(last updated 01-09-2022)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic
!pip install --upgrade joblib==1.1.0

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [ ]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [ ]:
len(docs)

18846

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-11-10 19:21:54,616 - BERTopic - Transformed documents to Embeddings
2022-11-10 19:22:34,522 - BERTopic - Reduced dimensionality
2022-11-10 19:23:09,103 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,6746,-1_to_the_is_and
1,0,1828,0_game_team_games_he
2,1,575,1_key_clipper_chip_encryption
3,2,525,2_ites_cheek_yep_huh
4,3,458,3_israel_israeli_jews_arab


In [ ]:
freq


,Topic,Count,Name
0,-1,6746,-1_to_the_is_and
1,0,1828,0_game_team_games_he
2,1,575,1_key_clipper_chip_encryption
3,2,525,2_ites_cheek_yep_huh
4,3,458,3_israel_israeli_jews_arab
...,...,...,...
215,214,10,214_1100_vhs_album_movies
216,215,10,215_memory_error_machine_register
217,216,10,216_carcinogenic_meat_carcinogens_smoke
218,217,10,217_article_buggy_reader_generated


In [ ]:
len(freq)

220

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(5)  # Select the most frequent topic

[('car', 0.01919315388699769),
 ('cars', 0.01565857868375224),
 ('mustang', 0.01518457082012263),
 ('ford', 0.012266255253911939),
 ('v8', 0.011999142966648809),
 ('engine', 0.011069327465226807),
 ('v6', 0.008428274182348379),
 ('wagon', 0.007429944058431178),
 ('audi', 0.006903384595969773),
 ('s4', 0.006791907279850261)]

**NOTE**: BERTopic is stocastic which means that the topics might differ across runs. This is mostly due to the stocastisc nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[0, 134, -1, 31, 82, -1, 34, 0, 0, -1]

In [ ]:
len(topic_model.topics_)

18846

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through a hundred topics to get a good 
understanding of the topics that were extracted. However, that takes quite some time and lacks a global representation. 
Instead, we can quantitatively visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [ ]:
probs.shape

(18846, 219)

In [ ]:
import random
random_document_id = random.randint(0, len(probs))
print(docs[random_document_id])
print('Topic: {}'.format(topic_model.topics_[random_document_id]))

# topic_model.visualize_distribution(probs[random_document_id], min_probability=0.015)
topic_model.visualize_distribution(probs[random_document_id], min_probability=0.003)


	Ladies & Gentlemen... Please keep this discussion in
	comp.os.ms-windows.advocacy where it belongs.
	Thank you.

John A. Grant						jagrant@emr1.emr.ca
Airborne Geophysics
Geological Survey of Canada, Ottawa
Topic: 133


## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=4)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.


In [ ]:
topic_model.visualize_heatmap(top_n_topics = 20, width=1000, height=1000)

You can set `n_clusters` in `visualize_heatmap` to order the topics by their similarity. This will result in blocks being formed in the heatmap indicating which clusters of topics are similar to each other. This step is very much recommended as it will make reading the heatmap easier.

In [ ]:
topic_model.visualize_heatmap(n_clusters = 3, top_n_topics = 20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank(log_scale=True)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs, n_gram_range=(2, 3))

In [ ]:
topic_model.get_topic(2)   # We select topic that we viewed before

[('huh lets', 0.48885266529056787),
 ('why ken huh', 0.48885266529056787),
 ('ken huh', 0.48885266529056787),
 ('ites yep', 0.48885266529056787),
 ('huh lets not', 0.48885266529056787),
 ('forget ites yep', 0.48885266529056787),
 ('forget ites', 0.48885266529056787),
 ('each why cheek', 0.48885266529056787),
 ('each why', 0.48885266529056787),
 ('cheek of why', 0.48885266529056787)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

2022-11-10 05:53:03,696 - BERTopic - Reduced number of topics from 222 to 61


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, -1, -1, 31, -1, -1, 34, 0, 0, -1, -1, -1, -1, -1, 11, 20, -1, 17, 39, -1, 6, -1, -1, -1, 0, 11, -1, -1, -1, 21, -1, -1, -1, 0, 30, 9, -1, -1, 7, -1, -1, 41, -1, -1, 0, -1, -1, -1, 1, -1, -1, 6, -1, 1, -1, -1, -1, -1, 23, -1, 0, -1, -1, -1, 20, -1, 0, -1, -1, -1, -1, -1, 35, -1, -1, -1, 0, 24, 4, 0, 52, 6, -1, -1, 33, -1, -1, 18, -1, -1, 0, 2, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1, 3, 2, 0, -1, -1, -1, -1, 9, -1, 10, 12, 20, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, 0, 31, 2, -1, 1, -1, -1, -1, 9, -1, -1, -1, 1, 9, 8, 0, 13, 7, 7, 31, -1, -1, -1, -1, 3, 10, 19, -1, 2, 36, -1, -1, -1, -1, -1, -1, 30, 22, 5, 27, -1, 4, -1, 0, -1, -1, 0, -1, 0, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, 2, 5, -1, 49, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, 4, -1, 50, 5, -1, 55, -1, -1, 7, 17, -1, -1, -1, 2, 0, 32, -1, 6, 1, -1, 1, 7, -1, 0, -1, -1, -1, 4, 0, -1, -1, 0, 0, -1, -1, -1, 30, 11, -1, 13, 2, 45, -1, 26, -1, -1, 4, -1, -1, -1, 36, -1, 28, -1, -1, -1, -1, -1, 37, -1,

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[5, 156, 76, 8, 70]

In [ ]:
topic_model.get_topic(156)

[('passenger', 0.036253424787563576),
 ('her', 0.03312297601207299),
 ('bike', 0.030951492817213405),
 ('ride', 0.025032767851855757),
 ('passengers', 0.02047649330714008),
 ('riding', 0.01886117007288508),
 ('she', 0.01653522391872404),
 ('shoulder', 0.016177613342846377),
 ('pillion', 0.013647994297031962),
 ('rider', 0.012698077157587294)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save model
topic_model.save("/content/drive/MyDrive/Colab Notebooks/CSEE5590-KDM/ICPs/ICP 11/my_model")	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
# Load model
my_model = BERTopic.load("/content/drive/MyDrive/Colab Notebooks/CSEE5590-KDM/ICPs/ICP 11/my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens") #Trained on NLI and STSB datasets

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

#Assignment: 
- Use another data source - Twitter / Reddit API
- Use another base model ([Here](https://www.sbert.net/docs/pretrained_models.html) is for a list of supported sentence transformers models).
- Compare and contrast topics with previously studied shallow learning technique LDA
